In [ ]:
# !pip install pyspark
# !pip install yfinance
pip install yahoo_fin

In [ ]:
# import pyspark
# from pyspark.sql import SparkSession
import pandas as pd
from datetime import datetime, timedelta
import json
from yahoo_fin.stock_info import get_data

In [ ]:
# THe following procedure is used to fetch stock data for a list of specified stock symbols from Yahoo Finance for the past week.

# Function to fetch historical stock data from Yahoo Finance
def fetch_historical_data(symbol):
    try:
        # Get historical data for the past week
        end_date = datetime.today().date()
        start_date = end_date - timedelta(days=7)
        data = get_data(symbol, start_date=start_date, end_date=end_date)
        return data
    except Exception as e:
        print(f"Failed to fetch data for symbol: {symbol}")
        print(e)
        return None

# Function to write stock data to JSON file
def write_to_json(data):
    with open('stocks.json', 'w') as f:
        json.dump(data, f, indent=4)

# Main function
def main():
    symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA', 'BRK-B', 'NVDA', 'JPM', 'JNJ',
               'V', 'PG', 'MA', 'UNH', 'HD', 'INTC', 'VZ', 'CRM', 'ADBE', 'PYPL']  # List of stock symbols
    stocks_data = {"stocks": []}

    for symbol in symbols:
        stock_data = fetch_historical_data(symbol)
        if stock_data is not None and not stock_data.empty:
            start_date = stock_data.index[0].strftime('%Y-%m-%d')  # Start date of the week
            end_date = stock_data.index[-1].strftime('%Y-%m-%d')  # End date of the week
            start_closing = round(stock_data.iloc[0]['close'], 2)  # Closing value at start of week
            end_closing = round(stock_data.iloc[-1]['close'], 2)  # Closing value at end of week
            stock_info_start = {
                "symbol": symbol,
                "date": start_date,
                "closing_value": start_closing
            }
            stock_info_end = {
                "symbol": symbol,
                "date": end_date,
                "closing_value": end_closing
            }
            stocks_data["stocks"].append(stock_info_start)
            stocks_data["stocks"].append(stock_info_end)
        else:
            print(f"No data available for symbol: {symbol}")

    write_to_json(stocks_data)

if __name__ == "__main__":
    main()


In [ ]:
# Read the CSV file
df_col = pd.read_csv('collaterals.csv')
df_cl = pd.read_csv('clients.csv')

# Add a new column 'value_past' and set it initially to NaN
df_col['value_past'] = float('nan')

# Load the JSON data
with open('stocks.json', 'r') as f:
    stock_data = json.load(f)

# Function to get the closing value for a stock asset yesterday
def get_closing_value_current(asset_name):
    for stock in stock_data['stocks']:
        # Convert the date string to a datetime object
        stock_date = datetime.strptime(stock['date'], '%Y-%m-%d').date()
        # Compare the dates
        if stock_date == datetime.now().date() - timedelta(days=1) and stock['symbol'] == asset_name:
            return stock['closing_value']
    return None

# Function to get the closing value for a stock asset 7 days ago
def get_closing_value_past(asset_name):
    for stock in stock_data['stocks']:
        # Convert the date string to a datetime object
        stock_date = datetime.strptime(stock['date'], '%Y-%m-%d').date()
        # Compare the dates
        if stock_date == datetime.now().date() - timedelta(days=7) and stock['symbol'] == asset_name:
            return stock['closing_value']
    return None

# Iterate over each row
for index, row in df_col.iterrows():
    # Check if the asset type is Stock
    if row['type'] == 'Stock':
        # Get the closing value for the asset
        closing_value_current = get_closing_value_current(row['asset_name'])
        if closing_value_current is not None:
            df_col.at[index, 'value_current'] = closing_value_current
        closing_value_past = get_closing_value_past(row['asset_name'])
        if closing_value_past is not None:
            df_col.at[index, 'value_past'] = closing_value_past

# Iterate over each row
for index, row in df_col.iterrows():
    # Check if the asset type is House, Car, or Office
    if row['type'] in ['House', 'Car', 'Office']:
        # If yes, set the value_past to the value_current
        df_col.at[index, 'value_past'] = row['value_current']
        df_col.at[index, 'fluctuation'] = 0
    else:
      df_col.at[index, 'fluctuation'] = ((row['value_current']  - row['value_past']) * 100) / row['value_past']

# Print the updated DataFrame
#print(df)

id_fluctuation = df_col[['id', 'asset_name', 'fluctuation']]
merged_df = pd.merge(df_cl, id_fluctuation, on='id')
print(merged_df)

# Write the merged DataFrame to a CSV file
merged_df.to_csv('merged_data.csv', index=False)




    id     profile       address corporate_information                 email  \
0    1  Individual   123 Main St                            john@example.com   
1    1  Individual   123 Main St                            john@example.com   
2    1  Individual   123 Main St                            john@example.com   
3    1  Individual   123 Main St                            john@example.com   
4    1  Individual   123 Main St                            john@example.com   
5    2     Company   456 Oak Ave              Fin Tech     susan@company.com   
6    2     Company   456 Oak Ave              Fin Tech     susan@company.com   
7    2     Company   456 Oak Ave              Fin Tech     susan@company.com   
8    2     Company   456 Oak Ave              Fin Tech     susan@company.com   
9    2     Company   456 Oak Ave              Fin Tech     susan@company.com   
10   2     Company   456 Oak Ave              Fin Tech     susan@company.com   
11   2     Company   456 Oak Ave        